# Migrate data from ToxRefDB to HAWC

Resource: User Guide (https://nepis.epa.gov/Exe/ZyPDF.cgi/P1015KWT.PDF?Dockey=P1015KWT.PDF)

In [ ]:
import os
import re
import string
from collections import defaultdict

import django
import pandas as pd
import psycopg2
import requests
from asgiref.sync import sync_to_async  # access the ORM safely within the notebook
from django.db.models import Q
from django.urls import reverse
from psycopg2.extras import DictCursor
from rapidfuzz import fuzz, process

In [ ]:
# update environment
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "hawc.main.settings.local")
django.setup()

In [ ]:
from hawc.apps.animalv2 import constants, models
from hawc.apps.assessment.models import DoseUnits, Species, Strain
from hawc.apps.study.models import Study
from hawc.apps.vocab.models import Guideline

### Import test studies

In [ ]:
# from hawc_client import HawcClient
# client = HawcClient("http://127.0.0.1:8000")
# client.authenticate("pm@hawcproject.org", "pw")

# hero_ids = [11805638, 11805639, 11805640, 11347133, 11347134,
#             11805642, 11347163, 11347164, 11347166, 11347167,
#             11805643, 11347168, 11805644, 11347169, 11347170,
#             11805645, 11347249, 11347258, 11816654, 11816658,
#             12330123, 12329953, 5932278, 12329803, 12329975, 12000793]

# for id in hero_ids:
#     response = client.study.create_from_identifier(
#         db_type="HERO", db_id=id, assessment_id=1
#     )
#     print("repsonse: ", response)

### Set up API access
https://hawc.readthedocs.io/latest/client/#api-access

In [ ]:
# login
session = requests.Session()

# base_url = https://hawcproject.org
base_url = "http://127.0.0.1:8000"

login = requests.post(
    f"{base_url}/user/api/token-auth/",
    json={"username": "admin@hawcproject.org", "password": "pw"},
    timeout=10,
)
print("login status:", login.json())
session.headers.update(Authorization=f"Token {login.json()['token']}")
login

In [ ]:
# define urls
get_study_url = reverse("study:api:study-list")
print("get_study_url:", get_study_url)
# animalv2
animalgroup_url = reverse("animalv2:api:animal-group-list")
chem_url = reverse("animalv2:api:chemical-list")
data_extraction_url = reverse("animalv2:api:data-extraction-list")
dose_group_url = reverse("animalv2:api:dose-group-list")
dose_response_group_level_data_url = reverse("animalv2:api:dose-response-group-level-data-list")
endpoint_url = reverse("animalv2:api:endpoint-list")
experiment_url = reverse("animalv2:api:experiment-list")
observation_time_url = reverse("animalv2:api:observation-time-list")
treatment_url = reverse("animalv2:api:treatment-list")

# vocab
toxrefdb_vocab_url = reverse("vocab:api:toxrefdb-nested")

### Set up database functions

In [ ]:
def connect_to_db(db_name, user, password, host, port):
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host, port=port)
    print(f"Connected to {db_name} successfully!")
    return conn


def fetch_data_from_source(query, conn):
    with conn.cursor() as cursor:
        cursor.execute(query)
        return cursor.fetchall()


def create_model_objects(url, data_list):
    responses = []
    for data in data_list:
        response = session.post(f"{base_url}/{url}", data)
        responses.append(response)
        if response.status_code != 201:
            print("status failed: ", response.json())

    return responses


def get_model_objects(url):
    response = session.get(f"{base_url}/{url}")
    return response


# set to toxref db
source_conn = connect_to_db("toxref", "hawc", "", "localhost", "5432")
cur = source_conn.cursor(cursor_factory=DictCursor)

### Create ToxRefDB Query

In [ ]:
# Study types are found in the User Guide (https://nepis.epa.gov/Exe/ZyPDF.cgi/P1015KWT.PDF?Dockey=P1015KWT.PDF)
# Data is split by study to avoid reading too much into memory at once.
# (The official downloadable ToxRefDB seems to be 66.5 MB, so this shouldn't be an issue.)
STUDY_TYPES = ["CHR", "SUB", "SAC", "DEV", "MGR", "REP", "DNT", "ACU", "OTH"]

# Define query for study dose-response data
## left join to include studies without tg or dose info
query = """SELECT * FROM prod_toxrefdb_2_1.chemical
LEFT JOIN prod_toxrefdb_2_1.study ON study.chemical_id=chemical.chemical_id
LEFT JOIN prod_toxrefdb_2_1.tg ON tg.study_id=study.study_id
LEFT JOIN prod_toxrefdb_2_1.dose ON dose.study_id=study.study_id
LEFT JOIN prod_toxrefdb_2_1.dtg ON dtg.tg_id=tg.tg_id AND dose.dose_id=dtg.dose_id
LEFT JOIN prod_toxrefdb_2_1.tg_effect ON tg.tg_id=tg_effect.tg_id
LEFT JOIN prod_toxrefdb_2_1.effect ON effect.effect_id=tg_effect.effect_id
LEFT JOIN prod_toxrefdb_2_1.endpoint ON endpoint.endpoint_id=effect.endpoint_id
LEFT JOIN prod_toxrefdb_2_1.dtg_effect ON tg_effect.tg_effect_id=dtg_effect.tg_effect_id AND dtg.dtg_id=dtg_effect.dtg_id
WHERE study.study_type = %s"""

chem_study_query = """SELECT * FROM prod_toxrefdb_2_1.chemical
LEFT JOIN prod_toxrefdb_2_1.study ON study.chemical_id=chemical.chemical_id
LEFT JOIN prod_toxrefdb_2_1.dose ON dose.study_id=study.study_id
WHERE study.study_type = %s;
"""

### Create Experiment and Chemical objects
Uses info from ToxRefDB study and chemical tables

In [ ]:
dsstox_array = []
dsstox_ids = []

# Read in all chemical dsstox_substance_ids
for type in STUDY_TYPES:
    cur.execute(chem_study_query, (type,))
    rows = cur.fetchall()
    dsstox = set(row["dsstox_substance_id"] for row in rows)
    dsstox_ids.extend(dsstox)

# Create new DSSTox objects if missing
dsstox_array.extend([{"dtxsid": value} for value in dsstox_ids])

print("### creating dsstox items ###")
responses = create_model_objects(reverse("assessment:api:dsstox-list"), dsstox_array)

In [ ]:
# map ToxRef design field values to HAWC constants
experiment_type_map = {
    "CHR": constants.ExperimentDesign.CH,
    "SUB": constants.ExperimentDesign.SB,
    "DEV": constants.ExperimentDesign.DV,
    "MGR": constants.ExperimentDesign.R2,  #  2-generation reproductive
    "REP": constants.ExperimentDesign.RP,
    "DNT": constants.ExperimentDesign.OT,  #  other
}

In [ ]:
# fuzzy study matching
hawc_studies = await sync_to_async(Study.objects.values_list)("id", "full_citation")
studies_list = await sync_to_async(list)(hawc_studies)
study_cit_map = [
    (id, "".join(char for char in cit if char not in string.punctuation))
    for id, cit in studies_list
]


async def get_hawc_study(trimmed_citation):
    # Try finding the study by title
    # Toxref format: J.C. Pettersen, A.D. Richter, and P.A. Gilles (1991) 90-Day Oral Toxicity Study in Rats. CIBA-GEIGY...
    # Title: 90-Day Oral Toxicity Study in Rats.
    title = re.search(r"\)\.?(.*?)([:.])", trimmed_citation)
    study = None
    if title and len(title.group(1)) > 25:
        title = title.group(1).strip()
        study = await sync_to_async(Study.objects.filter)(Q(full_citation__icontains=title))
        study = await sync_to_async(study.first)()
        print("study: ", study.full_citation if study else "None found by title")

    # If you can't access by title (format is not always consistent) try fuzzy match
    if not study:
        # remove punctuation and anything after "unpublished"
        trimmed_citation = "".join(
            char for char in trimmed_citation if char not in string.punctuation
        )
        trimmed_citation = (
            re.split(r"unpublished", trimmed_citation, flags=re.IGNORECASE)[0].strip().lower()
        )
        match, score, match_id = process.extractOne(
            trimmed_citation,
            [m[1].lower() for m in study_cit_map],
            scorer=lambda a, b, **kwargs: fuzz.ratio(a[:50], b[:50]),
        )
        # print("toxref citation: ", trimmed_citation)
        # print("match: ", match, " score: ", score, " match_id: ", match_id)
        if score >= 90:
            hawc_id = study_cit_map[match_id][0]
            study = await sync_to_async(Study.objects.get)(id=hawc_id)

    return study

In [ ]:
print("### Create a new HAWC Experiment and Chemical for each ToxRef study item ###")

study_to_exp_map = {}
toxref_to_hawc_chem_map = {}

for type in STUDY_TYPES:
    cur.execute(chem_study_query, (type,))
    rows = cur.fetchall()

    # Read into a df for data manipulation
    column_names = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=column_names)
    df = df.loc[:, ~df.columns.duplicated()]

    unique_studies = df["study_id"].unique()

    for study_id in unique_studies:
        # get the first row for each study
        row = df[df["study_id"] == study_id].iloc[0]

        # trimmed_citation = re.split(r'unpublished', toxref_citation, flags=re.IGNORECASE)[0].strip()
        # ToxRefDB study_citation does not map to 1-1 with HAWC Study full_citation. Get just the study title
        study = await get_hawc_study(row["study_citation"].strip())
        if not study:
            print("WARNING: No matching study found for citation:", row["study_citation"].strip())
            study = None

        guideline = await sync_to_async(Guideline.objects.filter)(
            Q(name__iexact=row["study_type_guideline"]) | Q(id=row["guideline_id"])
        )
        guideline = await sync_to_async(guideline.first)()

        # Create a new experiment object
        experiment_data = {
            "study": study.id if study else 1,
            "name": f"{row['dose_end']} {row['dose_end_unit']} {row['admin_route']} {row['study_type']} {row['preferred_name']}",  # Ex: 30 day oral CHR Isazofos
            "design": experiment_type_map[row["study_type"].upper()],
            "has_multiple_generations": True if row["study_type"] == "MGR" else False,
            "guideline": guideline.id if guideline else None,
            "comments": row["study_comment"] or "",
        }
        experiment = create_model_objects(experiment_url, [experiment_data])[0].json()
        # map ToxRefDB study to new  experiment
        # print("experiment: ", experiment)
        study_to_exp_map[row["study_id"]] = experiment.get("id")

        # Create a new HAWC Chemical for each ToxRef study (many-to-one study->chemical to one-to-one study->chemical)
        vehicle = row["vehicle"]
        if vehicle is None:
            inhalation_study = row["admin_route"].lower() == "inhalation"
            vehicle = "not reported, assumed clean air." if inhalation_study else "not reported"

        chemical_data = {
            "name": row["preferred_name"],
            "dtxsid_id": row["dsstox_substance_id"],
            "cas": row["casrn"],
            "experiment": experiment.get("id"),
            "source": row["substance_source_name"] or "",
            "purity": row["substance_purity"] or "",
            "comments": row["substance_comment"] or "",
            "vehicle": vehicle,
        }
        chemical = create_model_objects(chem_url, [chemical_data])[0].json()
        # print("chemical: ", chemical)
        toxref_to_hawc_chem_map[row["chemical_id"]] = chemical.get("id")

print(study_to_exp_map)
print(toxref_to_hawc_chem_map)

### Create HAWC Animalgroup objects

Uses info from ToxRefDB tg and study tables

In [ ]:
# map ToxRef field values to HAWC constants
life_stage_map = {
    "adult-pregnancy": constants.Lifestage.AG,
    "fetal": constants.Lifestage.DEV,  # verify
    "adult": constants.Lifestage.ADULT,
    "juvenile": constants.Lifestage.JUV,
    None: "",
}

route_exposure_map = {
    "oral": constants.RouteExposure.OR,
    "inhalation": constants.RouteExposure.I,
    "dermal": constants.RouteExposure.D,
}

variance_type_map = {
    "NA": constants.VarianceType.NA.label,
    "NR": constants.VarianceType.NR.label,
    "SD": constants.VarianceType.SD.label,
    "SE": constants.VarianceType.SE.label,
    "": constants.VarianceType.SD.label,
}

observation_time_map = {
    "week": constants.ObservationTimeUnits.WK.label,
    "lactation week": constants.ObservationTimeUnits.WK.label,
    "month": constants.ObservationTimeUnits.MON.label,
    "day": constants.ObservationTimeUnits.DAY.label,
    "LD": constants.ObservationTimeUnits.DAY.label,  # TODO: day vs lactation day?
    "GD": constants.ObservationTimeUnits.GD.label,
    "PND": constants.ObservationTimeUnits.PND.label,
}

treatment_related_map = {
    "true": constants.TreatmentRelatedEffect.YES.label,
    "false": constants.TreatmentRelatedEffect.NO.label,
}

Create a new HAWC treatment, animalgroup, species, and strain if needed for each ToxRef tg

In [ ]:
toxref_to_hawc_tg_map = {}
# toxref will have duplicate treatment groups (with differences that are irrelevant in hawc)
# we still need to identify them in order to create effects/etc. later
duplicate_tg_ids = defaultdict(list)


for type in STUDY_TYPES:
    cur.execute(query, (type,))
    rows = cur.fetchall()

    # Read into a df for data manipulation
    column_names = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=column_names)
    df = df.loc[:, ~df.columns.duplicated()]

    # avoid duplicate treatment groups by using all columns except 'dose_period'
    tg_cols = [
        "study_id",
        "sex",
        "generation",
        "dose_duration",
        "dose_duration_unit",
        "n",
        "tg_comment",
    ]
    tg_df = df[["tg_id", *tg_cols]]

    # store duplicate treatment groups
    for _, group in tg_df.groupby(
        ["study_id", "dose_duration", "dose_duration_unit", "n", "tg_comment"], dropna=False
    ):
        tg_ids = group["tg_id"].tolist()
        if len(tg_ids) >= 1:
            duplicate_tg_ids[tuple(tg_ids)] = tg_ids

    # filter unique treatment groups
    unique_tg_df = tg_df.drop(["sex", "generation"], axis=1)
    unique_tg_df = unique_tg_df.drop_duplicates(
        subset=[col for col in unique_tg_df.columns if col != "tg_id"]
    ).reset_index(drop=True)
    unique_tgs = unique_tg_df["tg_id"].unique()

    for tg_id in unique_tgs:
        row = df[df["tg_id"] == tg_id].iloc[0]
        experiment_id = study_to_exp_map[row["study_id"]]

        dose_duration = row["dose_duration"]
        if pd.isna(row["dose_duration"]):
            dose_duration = ""

        exposure_duration_description = f"{dose_duration} {row['dose_duration_unit']}"
        treatment_data = {
            "experiment": experiment_id,
            "name": f"{exposure_duration_description} {row['admin_route']} {row['preferred_name']}",
            "chemical_id": toxref_to_hawc_chem_map[row["chemical_id"]],
            "route_of_exposure": route_exposure_map[row["admin_route"].lower()],
            "exposure_duration": dose_duration,
            "exposure_duration_description": exposure_duration_description,
            "comments": row["tg_comment"] or "",
        }
        treatment = create_model_objects(treatment_url, [treatment_data])

        # add all treatment ids to map
        for tg_id_tuple in duplicate_tg_ids.keys():
            if tg_id in tg_id_tuple:
                for id in tg_id_tuple:
                    toxref_to_hawc_tg_map[id] = treatment[0].json().get("id")

    # create animal groups for each treatment group
    unique_tg_effects = tg_df.drop_duplicates(
        subset=[col for col in tg_df.columns if col != "tg_id"]
    ).reset_index(drop=True)
    unique_tgs = unique_tg_effects["tg_id"].unique()

    for tg_id in unique_tgs:
        row = df[df["tg_id"] == tg_id].iloc[0]
        experiment_id = study_to_exp_map[row["study_id"]]

        # create new animalgroup using tg and tg_effect
        if pd.notna(row["tg_effect_id"]):
            # create new species and strain if necessary
            species, created = await sync_to_async(Species.objects.get_or_create)(
                name=row["species"]
            )
            strain, created = await sync_to_async(Strain.objects.get_or_create)(
                name=row["strain"], species_id=species.id
            )

            if len(row["generation"]) > 2:
                gen = constants.Generation.F1  # default generation?
            else:
                gen = row["generation"]

            sex = constants.Sex.COMBINED if row["sex"] == "MF" else row["sex"]
            animalgroup_data = {
                "experiment": experiment_id,
                "name": f"{sex} {strain.name} {species.name}",  # ex: female wistar rat
                "sex": sex,
                "comments": row["tg_comment"] or "",
                "generation": gen,
                "lifestage_at_assessment": life_stage_map[row["life_stage"]],
                "lifestage_at_exposure": life_stage_map[row["life_stage"]],
                "species_id": species.id,
                "strain_id": strain.id,
            }
            create_model_objects(animalgroup_url, [animalgroup_data])[0].json()

print(toxref_to_hawc_tg_map)

Create HAWC dosegroup, endpoint, dataextraction, and dose_response_group_level_data from ToxRef dose, dtg, dtg_effect, tg, tg_effect

In [ ]:
# get toxrefdb vocab terms
toxrefdb_terms_df = pd.DataFrame(get_model_objects(toxrefdb_vocab_url).json())
toxrefdb_terms_df = toxrefdb_terms_df.rename(
    columns={
        "name_term_id": "name_term",
        "system_term_id": "system_term",
        "effect_term_id": "effect_term",
        "effect_subtype_term_id": "effect_subtype_term",
    }
)

In [ ]:
# arbitrary, unsure of the difference btw id and dose_id
dose_group_id = 13000
toxref_to_hawc_endpoint_map = {}
endpoint_to_term_map = {}
pd.set_option("future.no_silent_downcasting", True)

for type in STUDY_TYPES:
    cur.execute(query, (type,))
    rows = cur.fetchall()

    # Read into a df for data manipulation
    column_names = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=column_names)
    df = df.loc[:, ~df.columns.duplicated()]

    # filter unique doses
    unique_doses = df["dose_id"].unique()
    for dose_id in unique_doses:
        dose_df = df[df["dose_id"] == dose_id]

        # format data
        dose_df.loc[:, "effect_var_type"] = dose_df["effect_var_type"].fillna(
            constants.VarianceType.NA.label
        )
        dose_df.loc[:, "dtg_effect_comment"] = dose_df["dtg_effect_comment"].fillna("")
        dose_df.loc[:, "effect_val_unit"] = dose_df["effect_val_unit"].fillna(
            ""
        )  # set this to the default
        dose_df["critical_effect"] = (
            dose_df["critical_effect"]
            .fillna(False)
            .replace({"True": True, "False": False})
            .astype(bool)
        )

        # filter unique dose treatment groups and effects for each dose
        unique_dtg = dose_df["dtg_id"].unique()
        unique_tg_effect = dose_df["tg_effect_id"].unique()

        # create a hawc dosegroup for each toxrefdb dtg
        for dtg_id in unique_dtg:
            row = dose_df[dose_df["dtg_id"] == dtg_id].iloc[0]

            dose_adjusted = row["dose_adjusted"]
            if pd.isna(dose_adjusted):
                dose_adjusted = row["conc"] if not pd.isna(row["conc"]) else 0
                dose_unit, created = await sync_to_async(DoseUnits.objects.get_or_create)(
                    name="ppm"
                )
            else:
                dose_unit = await sync_to_async(DoseUnits.objects.get)(name="mg/kg/d")

            dosegroup_data = {
                "dose": dose_adjusted,
                "dose_group_id": dose_group_id,  # TODO: id vs dose_group_id?
                "dose_units": dose_unit,
                "treatment": toxref_to_hawc_tg_map[row["tg_id"]],
            }
            create_model_objects(dose_group_url, [dosegroup_data])[0].json()
            # increment manual dose group id
            dose_group_id += 1

        # filter unique treatment group effects
        for tg_effect_id in sorted(unique_tg_effect):
            dtg_effect_df = dose_df[dose_df["tg_effect_id"] == tg_effect_id]

            for dtg_effect_id in dtg_effect_df["dtg_effect_id"]:
                row = dtg_effect_df[dtg_effect_df["dtg_effect_id"] == dtg_effect_id].iloc[0]

                # get endpoint mapping
                endpoint_vocab = {
                    "name": row["effect_desc"],
                    "system": row["endpoint_category"],
                    "effect": row["endpoint_type"],
                    "effect_subtype": row["endpoint_target"],
                }
                # get term id info from existing toxrefdb df
                condition = (
                    toxrefdb_terms_df[list(endpoint_vocab)] == pd.Series(endpoint_vocab)
                ).all(axis=1)
                term_info = toxrefdb_terms_df[condition].iloc[0]
                endpoint_data = {"experiment": study_to_exp_map[row["study_id"]], **term_info}
                if (
                    f"{row['endpoint_id']} {study_to_exp_map[row['study_id']]}"
                    not in toxref_to_hawc_endpoint_map.keys()
                ):
                    # create a new endpoint
                    endpoint_id = (
                        create_model_objects(endpoint_url, [endpoint_data])[0].json().get("id")
                    )
                    toxref_to_hawc_endpoint_map[
                        f"{row['endpoint_id']} {study_to_exp_map[row['study_id']]}"
                    ] = endpoint_id
                    endpoint_to_term_map[
                        f"{row['endpoint_id']} {study_to_exp_map[row['study_id']]}"
                    ] = term_info["name_term"]
                else:
                    endpoint_id = toxref_to_hawc_endpoint_map[
                        f"{row['endpoint_id']} {study_to_exp_map[row['study_id']]}"
                    ]

                # Create new observation time model if needed
                if not pd.isna(row["time_unit"]):
                    obs_time_data = {
                        "observation_time": row["time"],
                        "observation_time_text": row["time"],
                        "observation_time_units": observation_time_map[row["time_unit"]],
                        "endpoint_id": endpoint_id,
                    }
                    obs_time = create_model_objects(observation_time_url, [obs_time_data])[0].json()

                    # if data extraction info is available, create model
                    data_extraction_data = {
                        "experiment": study_to_exp_map[row["study_id"]],
                        "treatment": toxref_to_hawc_tg_map[row["tg_id"]],
                        "endpoint": endpoint_id,
                        "dataset_type": constants.DatasetType.NOT_REPORTED,  # no toxrefdb equivalent
                        "method_to_control_for_litter_effects": constants.MethodToControlForLitterEffects.NR.label,  # no toxrefdb equivalent
                        "is_qualitative_only": False
                        if row["no_quant_data_reported"] == "f"
                        else True,
                        "variance_type": variance_type_map[row["effect_var_type"]],
                        "response_units": str(row["effect_val_unit"])[
                            :32
                        ],  # this is limited to 32 characters, unsure if we standardise somewhere
                        "dose_response_observations": row["effect_comment"] or "NA",
                        "observation_timepoint": obs_time.get("id"),
                        "result_details": f"{row['dtg_effect_comment']} {row['effect_comment']}",
                    }
                    data_extraction = create_model_objects(
                        data_extraction_url, [data_extraction_data]
                    )[0].json()

                    statistically_significant = (
                        constants.StatisticallySignificant.YES
                        if row["treatment_related"] == "t"
                        else constants.StatisticallySignificant.NO
                    )

                    # select appropriate dose column
                    dose_col = "dose_adjusted"
                    if pd.isna(dose_adjusted):
                        dose_col = "conc"
                    dose_adjusted = row[dose_col]

                    # calculate NOEL: lowest dose with a critical effect
                    filtered_df = dtg_effect_df[dtg_effect_df["critical_effect"]]
                    LOEL = filtered_df[dose_col].min()

                    # calculate LOEL: highest dose with no critical effect
                    filtered_df = dtg_effect_df[~dtg_effect_df["critical_effect"]]
                    NOEL = filtered_df[dose_col].max()

                    effect_val = row["effect_val"]
                    if pd.isna(effect_val):
                        effect_val = 0

                    dose_response_group_level_data = {
                        "data_extraction": data_extraction.get("id"),
                        "treatment_name": f"{row['dose_duration']} {row['dose_duration_unit']} {row['admin_route']} {row['preferred_name']}",  # from treatment model
                        "dose": dose_adjusted,  # TODO: check this out in terms mapping w dtg
                        "n": None if pd.isna(row["n"]) else row["n"],
                        "response": effect_val,  # TODO verify mapping and default
                        "variance": None if pd.isna(row["effect_var"]) else row["effect_var"],
                        "treatment_related_effect": treatment_related_map[
                            str(row["treatment_related"]).lower()
                        ],
                        "statistically_significant": statistically_significant,
                        "p_value": 0.05
                        if row["treatment_related"] == "t"
                        else "",  # TODO: ask about default p_value
                        "NOEL": round(NOEL)
                        if not pd.isna(NOEL)
                        else -999,  # HAWC requires ints not floats
                        "LOEL": round(LOEL)
                        if not pd.isna(LOEL)
                        else -999,  # HAWC requires ints not floats
                    }
                    create_model_objects(
                        dose_response_group_level_data_url, [dose_response_group_level_data]
                    )

## Migrate observation Data

In [ ]:
obs_query = (
    """SELECT study_id, endpoint_id, tested_status, reported_status FROM prod_toxrefdb_2_1.obs"""
)

obs_data = fetch_data_from_source(obs_query, source_conn)
# only migrate toxrefdb observations linked to an existing toxrefdb endpoint effect
obs_mapping = []

for row in obs_data:
    if f"{float(row[1])} {study_to_exp_map[row[0]]}" in endpoint_to_term_map.keys():
        obs_mapping.append(
            {
                "experiment_id": study_to_exp_map[row[0]],
                # key: endpoint_id and experiment_id
                "endpoint_id": endpoint_to_term_map[f"{float(row[1])} {study_to_exp_map[row[0]]}"],
                "tested_status": row[2],
                "reported_status": row[3],
            }
        )

for row in obs_mapping:
    print("row; ", row)
    object, created = await sync_to_async(models.Observation.objects.get_or_create)(**row)

In [ ]:
# Close the cursor and connection
# cur.close()
# source_conn.close()